### Load the required libraries

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
from pylab import *
%matplotlib inline

### Read the data frame from pickle file

In [ ]:
# Pickles locations
"""
http://mas-dse-open.s3.amazonaws.com/Weather/MEAN_SNWD.pickle
http://mas-dse-open.s3.amazonaws.com/Weather/MEAN_TRANGE.pickle
http://mas-dse-open.s3.amazonaws.com/Weather/MEAN_TAVG.pickle
http://mas-dse-open.s3.amazonaws.com/Weather/STAT_TAVG_RANGE.pickle 
"""


In [13]:
import pickle
Stats, desc=pickle.load(open('../../STAT_TAVG_RANGE.pickle','r'))
Stats2, desc=pickle.load(open('../../stat.pickle','r'))
MEAN_TAVG = pd.DataFrame(pickle.load(open('../../MEAN_TAVG.pickle','r')),columns=["station","mean"])
MEAN_TRANGE = pd.DataFrame(pickle.load(open('../../MEAN_TRANGE.pickle','r')),columns=["station","mean"])
MEAN_SNWD = pd.DataFrame(pickle.load(open('../../MEAN_SNWD.pickle','r')),columns=["station","mean"])

In [14]:
tavg = Stats['TAVG']['eigvec'][:,:3]
trange = Stats['TRANGE']['eigvec'][:,:3]
snwd = Stats2['SNWD']['eigvec'][:,:3]

In [48]:
Data_dir='../../../Data/Weather'
stations=pickle.load(open(Data_dir+'/stations.pkl','r'))
stations.reset_index(inplace=True)
stations.rename(columns={'index': 'station'}, inplace=True)
stations = stations[["station","latitude","longitude","elevation"]]

In [63]:
dist_from_coast = {}
with open("../../../Data/Weather/ghcnd-stations.txt","rb") as f:
    for line in f:
        info = line.strip().split()
        station = info[0]
        dist = float(info[-1])
        dist_from_coast[station] = dist

In [64]:
stations["dist_coast"] = stations['station'].map(dist_from_coast)

In [65]:
stations.head()

,station,latitude,longitude,elevation,dist_coast
0,ACW00011604,17.1167,-61.7833,10.1,0.410137
1,ACW00011647,17.1333,-61.7833,19.2,0.410137
2,AE000041196,25.3330,55.5170,34.0,9.094540
3,AF000040930,35.3170,69.0170,3366.0,1105.420000
4,AG000060390,36.7167,3.2500,24.0,4.694460


### Find the projections

In [35]:
MEAN_TAVG['TAVG_coeff'] = MEAN_TAVG['mean'].apply(lambda x: tavg.T.dot(np.nan_to_num(x))) 
MEAN_TRANGE['TRANGE_coeff'] = MEAN_TRANGE['mean'].apply(lambda x: trange.T.dot(np.nan_to_num(x))) 
MEAN_SNWD['SNWD_coeff'] = MEAN_SNWD['mean'].apply(lambda x: snwd.T.dot(np.nan_to_num(x))) 

In [ ]:
df = pd.merge(MEAN_TAVG,MEAN_TRANGE,on="station",how="inner")

In [38]:
df = df.merge(MEAN_SNWD,on="station",how="inner")

In [41]:
df.drop(["mean_x","mean_y","mean"], axis=1, inplace=True)

In [66]:
df = df.merge(stations,on="station",how="inner")

In [68]:
df.head()

,station,TAVG_coeff,TRANGE_coeff,SNWD_coeff,latitude,longitude,elevation,dist_coast
0,USC00044534,"[3047.96236332, 1974.34852034, 150.560792408]","[-2903.63287861, -236.907267527, 147.021790682]","[0.19150300062, 0.187262808215, -0.0401379552536]",36.0042,-119.9600,73.2,107.65500
1,USC00356784,"[2072.14900342, 880.454658675, -19.4039657277]","[-1588.34406546, 22.0915925777, 53.9057098222]","[0.315437798531, 0.126292083873, 0.792078997215]",42.7519,-124.5011,12.8,0.61097
2,USC00243581,"[949.76415135, 2361.83695166, 132.430208658]","[-2802.63818721, -165.774139321, 152.216161042]","[745.947251853, 256.091735413, 113.675894459]",47.1064,-104.7183,632.8,1316.54000
3,USC00205601,"[1447.7276104, 2119.09952327, 32.5684846715]","[-2354.34632034, -251.69735105, 150.704717534]","[506.947521322, 213.945709057, 136.385233577]",41.7500,-84.2167,247.2,685.50100
4,CA005010480,"[-107.474590032, 2456.37597386, 100.527486175]","[-2361.17001514, -121.515682174, 194.892166828]","[2731.8353828, 101.61002442, 328.650623536]",49.9170,-99.9500,409.0,908.27100


In [69]:
df.to_pickle("../../../Data/Weather/stations_projections.pickle")